#Векторизация входных и выходных данных
@Василий Гурьев

#Imports

In [0]:
import numpy as np
import pandas as pd
from google.colab import drive
import os

#Defs

##select_length

In [0]:
def select_length(data, minlen, maxlen): # ограничение длинны предложений
  return data.loc[(data.length >= minlen) & (data.length <= maxlen)]

#Loading

In [0]:
drive.mount('/content/gdrive') # подключаем диск
os.chdir('/content/gdrive/My Drive/Colab Notebooks/NNDW') # задаём рабочую директорию

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
data = pd.read_csv('data/sentences_2002.csv', usecols=['words']) # загрузка коллекции
data.words = data.words.apply(lambda words: eval(words.lower()))
# переводим строки в списки слов, приводя их к нижнему регистру
data['length'] = data.words.apply(len) # определяем длину предложений
data.head()

,words,length
0,"[как, сообщает, нтв, заокеанские, медики, реко...",17
1,"[по, их, словам, полезные, вещества, в, нем, п...",12
2,"[поход, в, сауну, как, средство, борьбы, с, по...",12
3,"[там, его, иногда, называют, русским]",5
4,"[запейте, их, двумя, большими, стаканами, воды]",6


In [0]:
data.shape

(155732, 2)

In [0]:
tags = pd.read_csv('data/tags_2002.csv', usecols=['tags']) # загружаем теги
tags.head()

,tags
0,"[['CONJ'], ['3per', 'VERB', 'impf', 'indc', 'p..."
1,"[['PREP'], ['ADJF', 'Apro', 'accs', 'neut', 's..."
2,"[['NOUN', 'inan', 'masc', 'nomn', 'sing'], ['P..."
3,"[['ADVB'], ['3per', 'NPRO', 'accs', 'masc', 's..."
4,"[['VERB', 'impr', 'intr', 'perf', 'plur'], ['3..."


In [0]:
data = pd.concat([data, tags], axis=1).dropna() # сопоставляем токены и теги

In [0]:
data.tags = data.tags.apply(lambda tags: eval(tags)) # переводим строки в списки

In [0]:
grammemes = pd.read_csv('dicts/grammemes.csv') # загружаем граммемы
grammemes.head()

,grammeme,description
0,NOUN,существительное
1,ADJF,полное прилагательное
2,ADJS,краткое прилагательное
3,COMP,компаратив
4,VERB,глагол


#Vectorization

In [0]:
minlen = 3 # минимальная длина предложения
maxlen = 30 # максимальная длина предложения

In [0]:
data30 = select_length(data, minlen, maxlen).sample(frac=1).reset_index(drop=True)
# выбираем предложения заданной длины, перемешиваем и сбрасываем индексы

In [0]:
data30['maxlen'] = data.words.apply(lambda words: max([len(word) for word in words]))
# получаем максимальную длину слов в предложениях

In [1]:
symbols = {chr(ord('а')+i): i + 1 for i in range(32)} # словарь символов
symbols['ё'] = 33
len(symbols)

33

In [0]:
x = np.zeros((len(data30), maxlen + 2, int(data30.maxlen.max() + 2)))
# задаём входной массив нужного размера с учётом начальных и конечных знаков токенов и предложений
for i, sentence in enumerate(data30.words): # в каждом прежложения
  for j, token in enumerate(sentence): # в каждом токене
    for k, char in enumerate(token): # для каждой буквы
      x[i, j+1, k+1] = symbols[char] # задаём индекс

In [0]:
y = np.zeros((len(data30), maxlen + 2, len(grammemes)))
# задаём выходной массив нужного размера с учётом начальных и конечных знаков предложений
for i, tags in enumerate(data30.tags): # для каждого предложения
  for j, tag in enumerate(tags): # для каждого тега
    for k, grammeme in enumerate(grammemes.grammeme): # перебираем все граммаемы
      if grammeme in tag: # и если они есть в теге
        y[i, j+1, k] = 1 # кодируем единицей

In [0]:
# true_tags = np.unique(y.reshape((-1, y.shape[-1])), axis=0) # уникальные теги

#Saving

In [0]:
np.savez_compressed('data/dataset_2002', x, y) # сохраняем векторизованный датасет

In [0]:
data30.to_csv('data/data_2002.csv', index=False) # и сырые данные

#END